In [2]:
import json
import re
import tqdm
from IPython.display import Markdown
import unidecode

## prepare raw wikiextract

In [2]:
%%bash
cat english.words| jq -s . > english.json

In [3]:
with open('english.json', 'r') as f:
    english_vocab_db = json.load(f)

In [4]:
i = 14
print(english_vocab_db[i]['word'])
print(len(english_vocab_db[i]['translations']))

cat
20


## helper functions to filter senses, synonyms and antonyms

In [5]:
def filter_senses(senses_list, word):
    filtered_senses = []
    for entry in senses_list:
        if 'glosses' in entry:
            glosses = entry['glosses']
            for gloss_entry in glosses:
                for gloss in gloss_entry.split(';'):
                    s = re.sub('\s+', ' ', re.sub("[\(\[].*?[\)\]]", "", gloss).replace(".", "")).strip()
                    if len(s) > 0 and word not in s:
                        filtered_senses.append(s)
    return filtered_senses

In [6]:
def filter_synonyms(synonyms_list):
    filtered_synonyms = []
    for entry in synonyms_list:
        if 'word' in entry:
            filtered_synonyms.append(entry['word'])
    return filtered_synonyms

In [7]:
def filter_antonyms(antonym_list):
    filtered_antonyms = []
    for entry in antonym_list:
        if 'word' in entry:
            filtered_antonyms.append(entry['word'])
    return filtered_antonyms

In [8]:
def add_wiktionary_item(item, db):
    item = {}
    if 'word' not in item.keys():
        return
    

## build a dictionary_database from the raw extract
using words as keys, enumerate those keys (e.g. `house`, `house1`, `house2` etc.) on duplicates

In [9]:
dictionary_database = {}
for item in tqdm.tqdm(english_vocab_db):
    if 'word' in item:
        word = item['word']
        i = 0
        while word in dictionary_database:
            i += 1
            word = f"{item['word']}{i}"
        dictionary_database[word] = item
        

100%|██████████| 1023195/1023195 [00:01<00:00, 818107.07it/s]


In [10]:
class NoDataException(Exception):
    pass

def get_attribute(word:str, attr:str, db = dictionary_database):
    if word not in db:
        raise NoDataException
    item = db[word]
    if attr not in item:
        raise NoDataException
    return item[attr]

def get_senses(word: str):
    
    senses = get_attribute(word, 'senses')
    
    return filter_senses(senses, word)

def get_synonyms(word: str):
    syns = get_attribute(word, 'synonyms')
    
    return filter_synonyms(syns)

def get_antonyms(word: str):
    ants = get_attribute(word, 'antonyms')
    
    return filter_antonyms(ants)

def display_info(word: str):
    
    if word not in dictionary_database:
        raise NoDataException
    
    try:
        senses = get_senses(word)
    except NoDataException:
        senses = []
    
    try:
        synonyms = get_synonyms(word)
    except NoDataException:
        synonyms = []
    
    try:
        antonyms = get_antonyms(word)
    except NoDataException:
        antonyms = []
        
    senses_str = '\n'.join([f"* {sense}" for sense in senses])
    
    synonyms_str = '\n'.join([f"* {syn}" for syn in synonyms])
    
    antonyms_str = '\n'.join([f"* {ant}" for ant in antonyms])
    
    markdown_string = f"""
## {word}

### Senses:

{senses_str}

### Synonyms:

{synonyms_str}

### Antonyms:

{antonyms_str}
    
    """
    
    return Markdown(markdown_string)

In [11]:
import random
list_words = []
for word in tqdm.tqdm(dictionary_database.keys()):
    item = dictionary_database[word]
    try:
        senses = get_senses(word)
        if len(senses) > 4:
            list_words.append(word)
            continue
        try:
            if len(get_antonyms(word)) >= 1:
                list_words.append(word)
                continue
        except NoDataException:
            if len(get_synonyms(word)) >= 1:
                list_words.append(word)
            
    except NoDataException:
        pass
    

100%|██████████| 1023195/1023195 [00:12<00:00, 84975.89it/s]


In [12]:
display(display_info("cat"))


## cat

### Senses:

* An animal of the family Felidae:
* A person:
* A sturdy merchant sailing vessel
* The game of "trap and ball"
* A vagina, a vulva
* the female external genitalia
* A double tripod with six feet, of which three rest on the ground, in whatever position it is placed
* A wheeled shelter, used in the Middle Ages as a siege weapon to allow assailants to approach enemy defences

### Synonyms:

* feliform
* feline cat
* pantherine cat
* panther
* Thesaurus:man
* Thesaurus:cat

### Antonyms:


    
    

----
## algorithm to build a grid of words

In [16]:
## store an optimized library version for production
optimized_lib = {}
for word in tqdm.tqdm(list_words):
    item = {}
    senses = None
    antonyms = None
    synonyms = None

    num_hints = 0

    raw_item = dictionary_database[word]
    raw_word = raw_item['word']

    def has_digit(s: str):
        for c in s:
            if c.isdigit():
                return True
        return False

    if has_digit(raw_word):
        continue

    try:
        senses = get_senses(word)

        new_senses = []

        for sense in senses:
            if raw_word.lower() in sense.lower():
                continue

            if sense.isupper():
                continue

            new_senses.append(sense)

        senses = new_senses

        num_hints += len(senses)

    except:
        pass
    try:
        synonyms = get_synonyms(word)

        num_hints += len(synonyms)
    except:
        pass
    try:
        antonyms = get_antonyms(word)

        num_hints += len(antonyms)
    except:
        pass

    if num_hints < 2:
        continue

    raw_item = dictionary_database[word]

    unaccented_key = unidecode.unidecode(word)
    unaccented_word = unidecode.unidecode(raw_item['word'])

    unaccented_key = unaccented_key.replace(" ", "").lower()
    unaccented_word = unaccented_word.replace(" ", "")

    
    item['senses'] = senses
    item['antonyms'] = antonyms
    item['synonyms'] = synonyms
    item['word'] = unaccented_word

    num_translations = 0
    if 'translations' in raw_item:
        num_translations = len(raw_item['translations'])
    
    if num_translations < 10:
        continue
    
    item['num_translations'] = num_translations

    optimized_lib[unaccented_key] = item

100%|██████████| 68842/68842 [00:01<00:00, 37266.20it/s]


In [17]:
len(optimized_lib)

21212

In [18]:
with open("en.json",'w') as f:
    json.dump(optimized_lib, f, indent = 4)

In [14]:
?json.dump

Signature:
json.dump(
    obj,
    fp,
    *,
    skipkeys=False,
    ensure_ascii=True,
    check_circular=True,
    allow_nan=True,
    cls=None,
    indent=None,
    separators=None,
    default=None,
    sort_keys=False,
    **kw,
)
Docstring:
Serialize ``obj`` as a JSON formatted stream to ``fp`` (a
``.write()``-supporting file-like object).

If ``skipkeys`` is true then ``dict`` keys that are not basic types
(``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
instead of raising a ``TypeError``.

If ``ensure_ascii`` is false, then the strings written to ``fp`` can
contain non-ASCII characters if they appear in strings contained in
``obj``. Otherwise, all such characters are escaped in JSON strings.

If ``check_circular`` is false, then the circular reference check
for container types will be skipped and a circular reference will
result in an ``OverflowError`` (or worse).

If ``allow_nan`` is false, then it will be a ``ValueError`` to
serialize out of range ``float`` 

In [28]:
import numpy as np
import random
from string import digits

class WordInfo(object):
    def __init__(self, word:str, y:int, x:int, is_vertical: bool, database=dictionary_database):
        self._dictionary_database = dictionary_database
        self._y = y
        self._x = x
        self._word = word
        self._hint = None
        self._is_vertical = is_vertical
        
        self.choose_info()
    
    def get_best_antonym(self) -> str:
        antonyms = get_antonyms(self._word)
        return random.choice(antonyms)
    
    def get_best_synonym(self) -> str:
        synonyms = get_synonyms(self._word)
        return random.choice(synonyms)
    
    def get_best_sense(self) -> str:
        senses = get_senses(self._word)
        return random.choice(senses)
        
    def choose_info(self):
        # first choose antonyms, then synonyms, then senses
        
        try:
            self._hint = f"opposite of {self.get_best_antonym()}"
        except NoDataException:
            pass
        
        try:
            self._hint = f"other word for {self.get_best_synonym()}"
        except NoDataException:
            pass
        
        self._hint = self.get_best_sense()
    
    def get_hint(self) -> str:
        return self._hint
    
    def get_hint_location(self):
        x = self._x if self._is_vertical else self._x - 1
        y = self._y if self._is_vertical else self._y - 1
        return (y, x)
    
    def is_vertical(self):
        return self._is_vertical

def create_word_grid(w: int, h: int, word_list = list_words, target_density = 0.5):
    grid = np.full(shape=(h,w), dtype=np.unicode, fill_value = ' ')
    
    locations = {}
    
    word_hints = {}
    
    def store_location(char: str, y: int, x: int):
        assert len(char) == 1
        
        if char not in locations:
            locations[char] = []
        
        locations[char].append([y,x])
        
    remove_digits = str.maketrans('', '', digits)
    n_words = len(list_words)
    
    def get_word(max_length: int, min_length = 0):
        assert max_length > 1
        
        index = random.randint(0,n_words-1)
        word = list_words[index][:]
        
        while len(word) >= max_length or not word.isalnum() or len(word) <= min_length:
            index = random.randint(0,n_words-1)
            word = list_words[index][:]
        
        return word
            
    def normalize_word(word:str):
        word = word.translate(remove_digits)
        return word.lower()
    
    def place_word(word:str, y: int, x:int, vertical:bool = False):
        normalized_word = normalize_word(word)
        n = len(normalized_word)
        if vertical:
            assert grid.shape[0] - n >= y
            for i, char in enumerate(normalized_word):
                grid[y + i,x] = char
                store_location(char, y+i, x)
        else:
            assert grid.shape[1] - n >= x
            for i, char in enumerate(normalized_word):
                grid[y,x + i] = char
                store_location(char, y, x+i)
        
        word_hints[normalized_word] = WordInfo(word, y, x, vertical)
        
        
    
    def density():
        return 1 - (grid == " ").sum() / (w * h)
    
    
    
    def check_if_fits(word:str, y:int, x:int, vertical:bool):
        n = len(word)
        if vertical:
            
            # check if there is space before and after
            if y - 1 >= 0 and grid[y - 1, x] != " ":
                return False
            if y + n < grid.shape[0] - 1 and grid[y+n,x] != " ":
                return False
            
            if grid.shape[0] - n < y or y < 0:
                #print("over board")
                return False
            
            for i, char in enumerate(word):
                char_x = x
                char_y = y + i
                
                if not (grid[char_y, char_x] == " " or grid[char_y, char_x] == char):
                    #print("not matching")
                    return False
                
                if grid[char_y, char_x] == " ":
                    # check for horizonatal neighbors:
                    if char_x - 1 >= 0 and grid[char_y, char_x - 1] != " ":
                        #print("3")
                        return False
                    if char_x + 1 < grid.shape[1] and grid[char_y, char_x + 1] != " ":
                        #print("4")
                        return False
        
        else:
            
            # check if there is space before and after
            if x - 1 >= 0 and grid[y, x - 1] != " ":
                return False
            if x + n < grid.shape[1] - 1 and grid[y,x + n] != " ":
                return False
            
            if grid.shape[1] - n < x or x < 0:
                #print("over board")
                return False
            
            for i, char in enumerate(word):
                char_x = x + i
                char_y = y
                
                if not (grid[char_y, char_x] == " " or grid[char_y, char_x] == char):
                    #print("not matching")
                    return False
                
                if grid[char_y, char_x] == " ":
                    # check for vertical neighbors:
                    if char_y - 1 >= 0 and grid[char_y - 1, char_x] != " ":
                        #print("1")
                        return False
                    if char_y + 1 < grid.shape[0] and grid[char_y + 1, char_x] != " ":
                        #print("2")
                        return False
        
        return True
        
    
    def get_crossover(word: str):
        # returns Tuple of: (y,x, is_vertical?) or None
        
        shuffled_order = list(range(len(word)))
        random.shuffle(shuffled_order)
        
        for index in shuffled_order:
            # check for existing locations
            char = word[index]
            if char in locations:
                char_locations = locations[char]
                
                for char_loc in char_locations:
                    # test vertical
                    y = char_loc[0] - index
                    x = char_loc[1]
                    
                    if check_if_fits(word, y, x, vertical=True):
                        return (y,x,True)
                    
                    # test horizontal
                    y = char_loc[0]
                    x = char_loc[1] - index
                    
                    if check_if_fits(word, y, x, vertical=False):
                        return (y,x,False)
        
        return None
    
    min_shape = min(w,h,30)
    
    # place first word:
    first_word = get_word(max_length=min_shape, min_length=min(10,grid.shape[1] - 2))
    
    # find random place:
    x = random.randint(0, grid.shape[1] - len(first_word) - 1)
    y = random.randint(0, grid.shape[0] - 1)
    
    place_word(first_word, y, x, vertical=False)
                
    i = 0
    
    
    current_density = density()
    
    while current_density < target_density:
        word = get_word(max_length=(1 - current_density ** 0.4) * min_shape,
                        min_length=max(min(10, 0.5 * (1 - current_density ** 0.3) * min_shape), 2))
        
        normalized_word = normalize_word(word)
        
        if normalized_word in word_hints:
            continue
        
        # check if matching characters exist:
        crossover = get_crossover(normalized_word)
        
        i += 1
        if i % 100000 == 0:
            print(i)
        if i > 100000:
            break
        
        if crossover == None:
            current_density = density()
            continue
        
        y,x,is_vertical = crossover
        
        place_word(word, y,x, is_vertical)
        
        current_density = density()
    
    print(i)
    return grid, word_hints
        
        
        
        
        
    

In [24]:
__file__


NameError: name '__file__' is not defined

In [29]:
#%%prun
gr, infos = create_word_grid(16*3,9*3, target_density=0.55)
def grid2mdown(grid):
    h,w  = grid.shape
    
    md_str = "     ".join(["|"] * (w+1)) + "\n"
    md_str += ":---:".join(["|"] * (w+1)) + "\n"
    
    for y in range(h):
        md_str += "|" + "|".join([(f"**{grid[y,x]}**" if grid[y,x] != " " else "     ") for x in range(w)]) + "|\n"
    
    return Markdown(md_str)

display(grid2mdown(gr))
    
    

762


|     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|     |     |     |     |     |**d**|**e**|**n**|**s**|**i**|**t**|**o**|**m**|**e**|**t**|**e**|**r**|     |     |**h**|     |     |**z**|     |     |     |**h**|**a**|**m**|**m**|**e**|**r**|**s**|     |     |**p**|**p**|**l**|     |     |     |     |**d**|     |**h**|**a**|**w**|**k**|
|     |     |     |**v**|     |     |     |**o**|     |     |     |     |     |**x**|     |     |**m**|     |**w**|**o**|**r**|**k**|**e**|**r**|**y**|     |     |**u**|     |     |     |     |     |     |**d**|     |**e**|     |**t**|**e**|**n**|**o**|**u**|**s**|     |**m**|     |     |
|     |     |**m**|**a**|**s**|**l**|**e**|**n**|**i**|**t**|**s**|**a**|     |     |     |     |     |     |     |**i**|     |     |**k**|     |     |     |**f**|**r**|**e**|**e**|     |**c**|**u**|**m**|**m**|**e**|**r**|     |     |**c**|     |     |**f**|     |     |**i**|     |**m**|
|     |     |     |**g**|     |     |     |**i**|     |**h**|     |     |**c**|     |     |**p**|**l**|**e**|**c**|**t**|**i**|**l**|**e**|     |     |**d**|     |**e**|     |     |**p**|     |**n**|     |**v**|     |**u**|     |**s**|**t**|**a**|**m**|**f**|**o**|**r**|**d**|     |**r**|
|     |     |     |**u**|**l**|**c**|**e**|**r**|**o**|**u**|**s**|     |**o**|     |**i**|     |**i**|     |**o**|     |     |     |     |**c**|     |**a**|**j**|**o**|**g**|     |**a**|     |**j**|     |     |     |**s**|     |     |**h**|     |     |     |     |     |**s**|**e**|**t**|
|     |**d**|**u**|**e**|     |     |     |**r**|     |**s**|     |**a**|**m**|**e**|**n**|     |**q**|     |**i**|     |**p**|**i**|**l**|**l**|**a**|**r**|     |**l**|     |     |**n**|     |**o**|     |     |**r**|**a**|**d**|**c**|**l**|**i**|**f**|**f**|**e**|     |**t**|     |     |
|     |**e**|     |     |     |**b**|     |**i**|     |     |**h**|     |**m**|     |**c**|     |**u**|     |**n**|     |**o**|     |     |**a**|     |**t**|     |**e**|     |     |**t**|     |**c**|     |**a**|     |**l**|     |     |**i**|     |     |**a**|     |**l**|     |**i**|     |
|     |**c**|**o**|**n**|**s**|**u**|**l**|**t**|**i**|**v**|**e**|     |**e**|**x**|**o**|**t**|**i**|**c**|     |**b**|**l**|**n**|     |**d**|     |**o**|     |     |**v**|     |**s**|**q**|**u**|**a**|**s**|**h**|     |**k**|**i**|**p**|     |     |**n**|     |**i**|     |**n**|     |
|**m**|**i**|     |     |     |**t**|     |**a**|     |     |**x**|     |**a**|     |**n**|     |**d**|     |     |     |**y**|     |**c**|     |**w**|**i**|**t**|**h**|**e**|     |     |     |**l**|     |**t**|     |     |     |     |**s**|     |**s**|**t**|**e**|**m**|     |**s**|     |
|     |**d**|     |**u**|     |**t**|**u**|**t**|     |**o**|     |     |**s**|     |**s**|     |**a**|     |**s**|     |**s**|     |**r**|**x**|     |**d**|     |     |**g**|**g**|     |     |**a**|     |     |**e**|     |**b**|     |**i**|     |     |**o**|     |**n**|     |**u**|     |
|     |**a**|     |**p**|     |**r**|     |**i**|**n**|**s**|**t**|**r**|**u**|**m**|**e**|**n**|**t**|**a**|**l**|**l**|**y**|     |**y**|     |     |     |     |     |**i**|     |**p**|**a**|**r**|**v**|**o**|**v**|**i**|**r**|**u**|**s**|     |     |**d**|     |**e**|**o**|**f**|     |
|     |**b**|**a**|**w**|**n**|**e**|**e**|**n**|     |**t**|     |     |**r**|     |**q**|     |**i**|     |**i**|     |**n**|     |**s**|**h**|**r**|**o**|**u**|**d**|**e**|**d**|     |**m**|     |**a**|     |**p**|     |**i**|     |     |     |**m**|     |     |**m**|     |**f**|     |
|     |**l**|     |**i**|     |**s**|     |**g**|**a**|**r**|**b**|     |**a**|     |**u**|     |**o**|     |**c**|**a**|**o**|     |**t**|     |**f**|     |     |     |     |     |**s**|**h**|**a**|**r**|**p**|     |**i**|**n**|**a**|**c**|**c**|**e**|**s**|**s**|**i**|**b**|**l**|**e**|
|     |**e**|     |**n**|     |**s**|     |     |     |**a**|     |     |**b**|**e**|**e**|     |**n**|     |**k**|     |**d**|     |**a**|     |**s**|     |**c**|     |**s**|     |     |**e**|     |     |**w**|     |     |**e**|     |**s**|     |**t**|     |     |**c**|     |**a**|     |
|     |     |**e**|**g**|**g**|     |     |**r**|**i**|**c**|**h**|     |**l**|     |**n**|**p**|     |**p**|**i**|     |**y**|**e**|**l**|**l**|     |**j**|**o**|**i**|**n**|**d**|**e**|**r**|     |     |     |**s**|     |     |     |**s**|     |**a**|     |**c**|     |     |**t**|     |
|     |**t**|     |     |     |     |**f**|     |     |**i**|     |     |**e**|     |**t**|     |**b**|     |**n**|     |     |     |**l**|     |**p**|     |**n**|     |**i**|     |     |**s**|     |**o**|     |**t**|     |     |**u**|     |**a**|**l**|**p**|**h**|**a**|**b**|**e**|**t**|
|     |**y**|     |     |**b**|**a**|**l**|**m**|**l**|**i**|**k**|**e**|     |**a**|     |     |**u**|**n**|**g**|**l**|**a**|**m**|**o**|**r**|**o**|**u**|**s**|     |**t**|     |     |**t**|     |**s**|**o**|**f**|**a**|     |**n**|     |     |**l**|     |**i**|     |     |     |**e**|
|     |**r**|     |     |**u**|     |**c**|     |     |**f**|     |**v**|     |**k**|     |     |**l**|     |     |     |     |     |**g**|     |**o**|     |**o**|     |     |**c**|     |     |     |**c**|     |**u**|     |     |**c**|**l**|**a**|**y**|     |**l**|**e**|**o**|**l**|**a**|
|**s**|**o**|**d**|     |**r**|     |     |**g**|**e**|**o**|**d**|**e**|**t**|**i**|**c**|**a**|**l**|     |     |     |**w**|     |**e**|     |**r**|     |**l**|     |**p**|**o**|**i**|**n**|**t**|**e**|     |     |**w**|     |**u**|     |     |     |     |**i**|     |     |**o**|     |
|**k**|     |**o**|     |**p**|**l**|     |     |     |**r**|     |**n**|     |     |     |**p**|     |     |**b**|**g**|**a**|     |**n**|     |     |**s**|**i**|**c**|     |**n**|     |     |     |**o**|     |     |**r**|     |**t**|     |     |**d**|**o**|**a**|**t**|**i**|**n**|**g**|
|**i**|     |**w**|     |     |     |     |**s**|**i**|**m**|     |     |**n**|**a**|**g**|**p**|**u**|**r**|     |     |**l**|     |     |**w**|     |     |**d**|     |     |**t**|     |**w**|**i**|**l**|**d**|**f**|**i**|**r**|**e**|     |**e**|     |     |**s**|     |     |**g**|     |
|**r**|     |**n**|     |     |**f**|**d**|**m**|     |     |**p**|**s**|     |     |     |**e**|     |     |     |     |**d**|**e**|**v**|**a**|**s**|**t**|**a**|**t**|**i**|**o**|**n**|     |     |**a**|     |     |**g**|     |     |**b**|**l**|**o**|**o**|**t**|**h**|     |**v**|     |
|**r**|**e**|**s**|**e**|**t**|     |     |**e**|     |     |**i**|     |     |     |     |**r**|**o**|**u**|**p**|     |**r**|     |     |**t**|     |     |**t**|     |     |**r**|     |     |     |     |**p**|**i**|**g**|**g**|**y**|     |**s**|     |     |     |**d**|     |**i**|     |
|     |     |**t**|     |**i**|     |**c**|**a**|**p**|**i**|**t**|**a**|**l**|**i**|**s**|**t**|     |     |**r**|     |**o**|     |     |**t**|     |**b**|**e**|**a**|**s**|**t**|     |     |     |**h**|     |     |**l**|     |     |     |**e**|**n**|**r**|**o**|**l**|**l**|**e**|**d**|
|**m**|**e**|**a**|**s**|**l**|**y**|     |**t**|     |     |**k**|     |     |**d**|     |**a**|**l**|**l**|**y**|     |**n**|**c**|     |**l**|     |**e**|     |     |**o**|     |**m**|     |**w**|**a**|**n**|**t**|**e**|**d**|     |     |     |     |**a**|     |     |     |**w**|     |
|**e**|     |**g**|     |**l**|     |     |**o**|     |**p**|**i**|**p**|     |**p**|     |**i**|     |     |     |     |     |     |     |**e**|     |**n**|     |     |**p**|     |**s**|     |     |**n**|     |     |**r**|     |     |     |**a**|     |**c**|     |     |     |     |     |
|     |**k**|**e**|**y**|**p**|**o**|**i**|**n**|**t**|     |**n**|     |**g**|**r**|**a**|**n**|**d**|**f**|**a**|**t**|**h**|**e**|**r**|**l**|**y**|     |     |**w**|**a**|**r**|**b**|**i**|**r**|**d**|     |     |     |     |     |     |**c**|**l**|**e**|**a**|**n**|     |     |     |


In [279]:
i=220

word = list(infos.keys())[i]

print(word, "--", infos[word].get_hint())

edie -- surname


In [360]:
display(display_info("wagenboom"))


## wagenboom

### Senses:

* A South African proteaceous tree
* The tough wood of this tree, used for making wagon wheels

### Synonyms:

* waboom
* wagon tree

### Antonyms:


    
    

In [30]:
from enum import Enum

class HintOrientation(Enum):
    VERTICAL = 0
    HORIZONTAL = 1
    BOTH = 2
    
class FieldType(Enum):
    EMPTY = 0
    HINT = 1
    LETTER = 2

class Field(object):
    def __init__(self, field_type: FieldType = FieldType.EMPTY):
        self._field_type = field_type
    
    def get_type(self) -> FieldType:
        return self._field_type
    
    def get_content(self) -> str:
        return None

class HintField(Field):
    def __init__(self, horizontal_hint: str = None, vertical_hint: str = None):
        super().__init__(field_type=FieldType.HINT)
        
        self._horizontal_hint = horizontal_hint
        self._vertical_hint = vertical_hint
    
    def get_horizontal_hint(self) -> str:
        return self._horizontal_hint
    
    def get_vertical_hint(self) -> str:
        return self._vertical_hint
    
    def set_horizintal_hint(hint: str):
        self._horizontal_hint = hint
    
    def set_vertical_hint(hint:str):
        self._vertical_hint = hint

class LetterField(Field):
    def __init__(self, letter: str):
        assert len(letter) <= 1
        
        super().__init__(field_type = FieldType.LETTER)
        
        self._letter = letter
        self._revealed = False
    
    def get_content(self) -> str:
        return self._letter
    
    def reveal(self):
        self._revealed = True
    
    def is_revealed(self) -> bool:
        return self._revealed

class Grid(object):
    def __init__(self, width: int, height: int, density = 0.5):
        self._width = width
        self._height = height
        self._density = density
        self._grid = []
        self._build_grid()
    
    def _build_grid(self):
        raw_grid, word_infos = create_word_grid(self._width - 1, self._height - 1, target_density=self._density)
        
        # note: we will append an additional row and column, to have enough space to place hint fields
        
        self._grid = [[Field()] * self._width] # initialize with empty row
        
        for y in range(self._height):
            row = [Field()] # initialize row with empty column
            for x in range(self._width):
                raw_cell = raw_grid[y-1,x-1]
                if raw_cell == " ":
                    row.append(Field())
                else:
                    row.append(LetterField(raw_cell))
            
            self._grid.append(row)
        
        # place hint fields:
        for word, info in word_infos.items():
            y,x = info.get_hint_location()
            # correct offset
            y += 1
            x += 1
            
            cell = self._grid[y][x]
            
            # check if we already have a hint here:
            if cell.get_type() == FieldType.HINT:
                if info.is_vertical():
                    cell.set_vertical_hint(info.get_hint())
                else:
                    cell.set_horizintal_hint(info.get_hint())
            elif cell.get_type() == FieldType.LETTER:
                # edge case: a word has "eaten up" another one, skipping that case
                pass
            
            else:
                if info.is_vertical():
                    self._grid[y][x] = HintField(vertical_hint=info.get_hint())
                else:
                    self._grid[y][x] = HintField(horizontal_hint=info.get_hint())

                        
class Crossword(object):
    def __init__(self, width: int, height: int, database: dict):
        self._width = width
        self._height = height
        self._database = database
        self._grid = Grid(width, height)
    
    
        
        

In [31]:
cw = Crossword(16*3, 9*3, english_vocab_db)

209


In [33]:
cw._grid._grid

[[<__main__.HintField at 0x7f207d65a820>,